<a href="https://colab.research.google.com/github/oxanaRC/CE902/blob/main/Constructing_Historic_(4mnths)_Median_Sales_Price_per_Postal_Area_and_per_bedroom_030821.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

03.08.21
Identifying PostCode areas in the property sales dataset (all) and obtaining historic (4mnths) Median Sales Price per bedroom per PostCode area. 






In [1]:
!pip install harperdb

In [2]:
import harperdb
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
url='https://ce901data-orc0770.harperdbcloud.com'
db=harperdb.HarperDB(url=url,
            username='myuser',
            password='abcdef',
            timeout=500)

In [4]:
db

Below data is extracted from the SQL database, however because of poor performance of the JOIN SQL operations, instead of relational SQL statements, the JOIN operations are performed in pandas DataFrames (according to https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html).


The data extraction from SQL tables in placed within the loop to avoid the database timeout issues.


In [5]:
list_of_dfs = {}
count=0
for mt in range(1, 11):
  for yr in range (2018,2021):
    year=yr
    month=mt
    count=count+1
    #sqlstring1='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date '
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE (num_of_beds>0 and deed_date LIKE "%{}/{}")'
    #sqlstring2='FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE deed_date LIKE "%{}/{}"'
    #sqlstring=sqlstring1+sqlstring2
    sqlstring='SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%{}/{}"'
    sqlstring=sqlstring.format(month,year)
    print(sqlstring)
    my_db_data=db.sql(sqlstring)
    list_of_dfs[count] = pd.DataFrame(my_db_data)
    print(list_of_dfs[count].shape)
df_prop_sales = pd.concat(list_of_dfs, sort=False)
#excluding 'other' property types (i.e. commercial) from the properties sales data 
df_prop_sales=df_prop_sales[(df_prop_sales['property_type']!='O')]
df_prop_sales

SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%1/2018"
(169065, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%1/2019"
(159110, 8)
SELECT propertypurchases.unique_id, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type, propertypurchases.deed_date, propertypurchases.postcode FROM projectdata.propertypurchases WHERE deed_date LIKE "%1/2020"
(133520, 8)
SELECT propertypurchases.unique_id, propertypurchase

unique_id  price_paid  ...   deed_date postcode
1  0      64342BFD-ABA7-422C-E053-6C04A8C0FB8A     1740000  ...  11/01/2018   E8 4NE
   1      64342BFD-ABAA-422C-E053-6C04A8C0FB8A     1525000  ...  19/01/2018   N5 1BZ
   2      64342BFD-ABAB-422C-E053-6C04A8C0FB8A     3306000  ...  17/01/2018  W11 1NZ
   3      64342BFD-ABAC-422C-E053-6C04A8C0FB8A      442000  ...  04/01/2018  IG3 9NB
   4      64342BFD-ABB0-422C-E053-6C04A8C0FB8A     1375000  ...  12/01/2018  W12 8JN
...                                        ...         ...  ...         ...      ...
30 84973  C18F412B-BE2B-81A6-E053-6B04A8C0AD18      125000  ...  09/10/2020  LS9 9DA
   84974  C18F412B-BEA1-81A6-E053-6B04A8C0AD18      147500  ...  06/10/2020  HD5 9NY
   84975  C18F412B-BEA2-81A6-E053-6B04A8C0AD18      145000  ...  06/10/2020  HD5 9NY
   84977  C18F412B-BEA5-81A6-E053-6B04A8C0AD18      104000  ...  02/10/2020  WF4 6AB
   84979  C18F412B-BEE8-81A6-E053-6B04A8C0AD18      169000  ...  30/10/2020  WF7 5PG

[2563547 rows x 8 columns]

In [6]:
sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds FROM projectdata.bedrooms WHERE num_of_beds>0'
bedrooms=db.sql(sqlstring)
df_bedrooms=pd.DataFrame(bedrooms)
df_bedrooms.rename(columns = {'id':'unique_id'}, inplace = True)
df_bedrooms

,unique_id,num_of_beds
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1
4,8355F008-F016-55C5-E053-6B04A8C0D090,1
...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81


In [7]:
#sqlstring='SELECT bedrooms.id, bedrooms.num_of_beds, propertypurchases.price_paid, propertypurchases.town, propertypurchases.district, propertypurchases.estate_type, propertypurchases.property_type FROM projectdata.bedrooms LEFT OUTER JOIN projectdata.propertypurchases ON bedrooms.id=propertypurchases.unique_id WHERE num_of_beds>0'
#my_db_data=db.sql(sqlstring)
#data=pd.DataFrame(my_db_data)
data = pd.merge(df_bedrooms, df_prop_sales, how="left", on="unique_id")
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [8]:
#check rows where postcode is NAN - these are the result of rows added on the Join with bedrooms data where bedrooms data is for prop sale which happened after the considered period
#data_t=data[ (data['postcode'].isnull()) ]
#drop NAN rows from data
data=data.dropna(subset=['postcode'])     
data 

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [9]:
#checking duplicates (some bedroom data was loaded twice by mistake)
duplicate = data[data.duplicated()]
duplicate

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
475,79A74E22-514F-1289-E053-6B04A8C01627,1,155000.0,WARWICK,WARWICK,L,F,03/10/2018,CV34 4NL
1658,79A74E21-9309-1289-E053-6B04A8C01627,1,185000.0,WELWYN GARDEN CITY,WELWYN HATFIELD,L,F,28/09/2018,AL7 3XA
3901,6B32222C-B011-01F1-E053-6C04A8C0D2C8,1,97000.0,SHEFFIELD,SHEFFIELD,L,F,21/02/2018,S13 7BW
4256,85866A65-48BA-143F-E053-6B04A8C06A15,1,284000.0,LONDON,LEWISHAM,L,F,07/12/2018,SE6 4RL
5129,7E86B6FB-4182-458C-E053-6B04A8C0C84C,1,220000.0,SUTTON,SUTTON,L,F,11/12/2018,SM1 4PH
...,...,...,...,...,...,...,...,...,...
1230818,7C2D0701-1FEB-4963-E053-6B04A8C07B97,8,730000.0,LONDON,GREENWICH,F,S,07/11/2018,SE9 3NL
1230980,7C2D0701-2821-4963-E053-6B04A8C07B97,8,331000.0,BIRMINGHAM,BIRMINGHAM,F,T,26/10/2018,B29 7DL
1231259,773788C3-5811-2CE4-E053-6C04A8C05E57,9,217000.0,COVENTRY,COVENTRY,F,T,07/09/2018,CV4 8FR
1231442,79A74E22-5879-1289-E053-6B04A8C01627,10,1925000.0,BIRMINGHAM,BIRMINGHAM,F,D,10/10/2018,B15 3RE


In [10]:
data=data.drop_duplicates()
data

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD
...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL


In [11]:
#data preparation: 1. new column: postcode area with month and year and (eg. NR3_04_2018) 2. new column: calculating price per bedroom (price paid/number of bedrooms)
data_prep=data
data_prep['postcode_area_m_y']=data_prep['postcode'].str.split(' ').str[0]+'_'+data_prep['deed_date'].str.split('/').str[1]+'_'+data_prep['deed_date'].str.split('/').str[2]
data_prep['price_per_bedroom']=data_prep['price_paid']/data_prep['num_of_beds']
data_prep

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,M30_07_2018,100000.000000
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,M25_11_2019,79500.000000
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,SW12_02_2020,314000.000000
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,CM7_02_2019,170000.000000
...,...,...,...,...,...,...,...,...,...,...,...
1231571,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,WN7_02_2019,1680.851064
1231572,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,NP12_04_2019,1104.166667
1231573,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,CH6_02_2019,1272.727273
1231574,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,OL14_04_2019,1172.839506


In [12]:
#for testing calculations for a specific area
df_co4=data_prep[(data_prep['postcode_area_m_y']=='CO4_10_2018')]
co4_median_price_per_bed = df_co4.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
co4_median_price_per_bed

,postcode_area_m_y,price_per_bedroom
0,CO4_10_2018,98333.333333


In [13]:
#function which will be used to work as follows -> e.g. we have 'postcode_area_m_y' M30_07_2018 - passing this as parameter to the setFourMonthsPlus(M30,7,2018) - function returns 'M30_11_2018)
def setFourMonthsPlus(anId,aMonth,aYear):
  if aMonth>=9:
    tMonth=aMonth+4-12
    tYear=aYear+1
  else:
    tMonth=aMonth+4
    tYear=aYear
  myNewId= str(anId).split('_')[0]+'_'+str(tMonth).zfill(2) +'_'+str(tYear)
  return myNewId

In [14]:
#creating new dataframe which stores median price_per_bedroom for each postcode_area_m_y
g_median_price_per_bed = data_prep.groupby(['postcode_area_m_y'])['price_per_bedroom'].median().reset_index()
g_median_price_per_bed['m_y']=g_median_price_per_bed['postcode_area_m_y'].str[-7:]
g_median_price_per_bed

,postcode_area_m_y,price_per_bedroom,m_y
0,AL10_01_2018,117916.666667,01_2018
1,AL10_01_2019,120000.000000,01_2019
2,AL10_01_2020,120500.000000,01_2020
3,AL10_02_2018,121666.666667,02_2018
4,AL10_02_2019,125000.000000,02_2019
...,...,...,...
55894,YO8_10_2019,63000.000000,10_2019
55895,YO8_11_2018,62000.000000,11_2018
55896,YO8_11_2019,66000.000000,11_2019
55897,YO8_12_2018,63341.666667,12_2018


In [15]:
#Need to add the following assumption: if data belongs to 1/2018, 2/2018, 3/2018 and 4/2018 - we do not have historic correspondong data for 09/2017, 10/2017, 11/2017 and 12/2017,
#so the working assumption is that median prices per bedroom in 09/2017 were the same as in 1/2018, median prices per bedroom in 10/2017 were the same as in 2/2018, median prices per bedroom in 11/2017 were the same as in 3/2018.
#To implement this assumption - we'll be getting the section of the above table for  1/2018, 2/2018, 3/2018 and 4/2018 so this will be appended to the new table as price_per_bedroom_4mnths_ago
df_add_assumption=g_median_price_per_bed[(g_median_price_per_bed['m_y']=='01_2018')|(g_median_price_per_bed['m_y']=='02_2018')|(g_median_price_per_bed['m_y']=='03_2018')|(g_median_price_per_bed['m_y']=='04_2018')]
df_add_assumption.rename(columns = {'price_per_bedroom':'median_ppb_4mnths_ago'}, inplace = True)
df_add_assumption=df_add_assumption.drop(['m_y'], axis = 1)
df_add_assumption

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,postcode_area_m_y,median_ppb_4mnths_ago
0,AL10_01_2018,117916.666667
3,AL10_02_2018,121666.666667
6,AL10_03_2018,118666.666667
8,AL10_04_2018,131666.666667
26,AL1_01_2018,189083.333333
...,...,...
55855,YO7_04_2018,75000.000000
55873,YO8_01_2018,63500.000000
55876,YO8_02_2018,61000.000000
55879,YO8_03_2018,55333.333333


In [16]:
# looping through each row in the g_median_price_per_bed and obtaining 'four months plus' value for each postcode_area_m_y,
# for example for the g_median_price_per_bed row which contains values : 'AL10_01_2018, 117916.66' the list below will contain 'AL10_05_2018, 117916.66' 
# this means that for the postal area AL10 in April 2018, the median price per bedroom was 117916.66 four months ago.
list_of_months_plus_four=[]
for row in g_median_price_per_bed.itertuples():
   myMonth=int(str(row[1]).split('_')[1])
   myYear =int((str(row[1])).split('_')[2])
   fourMnthsPlus=setFourMonthsPlus(row[1],myMonth,myYear)
   new_row=[]
   new_row.append(fourMnthsPlus)
   new_row.append(row[2])
   list_of_months_plus_four.append(new_row)


In [17]:
#data from the list above is placed in the g_median_price_per_bed dataframe (it overwrites the data which was there before)
g_median_price_per_bed = pd.DataFrame(list_of_months_plus_four)
g_median_price_per_bed.columns =[ 'postcode_area_m_y','median_ppb_4mnths_ago', ]
g_median_price_per_bed

,postcode_area_m_y,median_ppb_4mnths_ago
0,AL10_05_2018,117916.666667
1,AL10_05_2019,120000.000000
2,AL10_05_2020,120500.000000
3,AL10_06_2018,121666.666667
4,AL10_06_2019,125000.000000
...,...,...
55894,YO8_02_2020,63000.000000
55895,YO8_03_2019,62000.000000
55896,YO8_03_2020,66000.000000
55897,YO8_04_2019,63341.666667


In [18]:
#combining two dataframes: g_median_price_per_bed (historic [4m] median prices per bedrooms  for postal areas starting from 04_2018) and df_add_assumption (as we decided about the assumption for 01_2018, 02_2018, 03_2018, 04_2018) 
g_median_price_per_bed_4m_ago = pd.concat([df_add_assumption, g_median_price_per_bed], axis=0)
g_median_price_per_bed_4m_ago

,postcode_area_m_y,median_ppb_4mnths_ago
0,AL10_01_2018,117916.666667
3,AL10_02_2018,121666.666667
6,AL10_03_2018,118666.666667
8,AL10_04_2018,131666.666667
26,AL1_01_2018,189083.333333
...,...,...
55894,YO8_02_2020,63000.000000
55895,YO8_03_2019,62000.000000
55896,YO8_03_2020,66000.000000
55897,YO8_04_2019,63341.666667


In [19]:
  #downloading csv file with g_median_price_per_bed_4m_ago data, so the median priced per bedroom data could be kept in a db
  from google.colab import files
  m_from='01_2018'
  m_to='02_20'
  my_file='median_price_per_bedroom_four_months_ago_period_{}_{}.csv'.format(m_from, m_to)
  with open(my_file, 'w') as f:
    g_median_price_per_bed_4m_ago.to_csv(my_file, index=False)
  files.download(my_file)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#Now mergin the sales data with g_median_price_per_bed_4m_ago, so each property sale transaction row contains 'median_ppd_4_mnths_ago'
data_prep = pd.merge(data_prep, g_median_price_per_bed_4m_ago, how="left", on="postcode_area_m_y")
data_prep

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,median_ppb_4mnths_ago
0,726BF13A-B9B6-0A46-E053-6C04A8C01D0D,1,100000.0,MANCHESTER,SALFORD,L,T,20/07/2018,M30 8EG,M30_07_2018,100000.000000,56187.500000
1,9B361206-34DE-1904-E053-6B04A8C0EEB5,1,79500.0,MANCHESTER,BURY,L,S,08/11/2019,M25 1ET,M25_11_2019,79500.000000,79000.000000
2,726BF13B-13CE-0A46-E053-6C04A8C01D0D,1,128000.0,NORWICH,NORWICH,L,F,20/04/2018,NR3 1SX,NR3_04_2018,128000.000000,64341.666667
3,A71375FD-EBA1-7576-E053-6C04A8C0462F,1,314000.0,LONDON,LAMBETH,L,F,21/02/2020,SW12 0JP,SW12_02_2020,314000.000000,330000.000000
4,8355F008-F016-55C5-E053-6B04A8C0D090,1,170000.0,BRAINTREE,BRAINTREE,L,F,01/02/2019,CM7 1TD,CM7_02_2019,170000.000000,101666.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
1219325,8355F009-2B4F-55C5-E053-6B04A8C0D090,47,79000.0,LEIGH,WIGAN,L,T,08/02/2019,WN7 1NG,WN7_02_2019,1680.851064,40499.375000
1219326,8A78B2B0-784A-5CB0-E053-6B04A8C0F504,48,53000.0,BLACKWOOD,CAERPHILLY,F,S,12/04/2019,NP12 3LQ,NP12_04_2019,1104.166667,50500.000000
1219327,8355F009-8A15-55C5-E053-6B04A8C0D090,55,70000.0,FLINT,FLINTSHIRE,F,T,11/02/2019,CH6 5DY,CH6_02_2019,1272.727273,45416.666667
1219328,8A78B2B0-4326-5CB0-E053-6B04A8C0F504,81,95000.0,TODMORDEN,CALDERDALE,L,T,08/04/2019,OL14 8NL,OL14_04_2019,1172.839506,56650.000000


In [24]:
#for testing calculations for a specific area
df_co4_t=data_prep[(data_prep['postcode_area_m_y']=='CO4_02_2019')]
co4_median_price_per_bed_t = df_co4_t.groupby(['postcode_area_m_y'])['median_ppb_4mnths_ago'].median().reset_index()
df_co4_test=pd.concat([co4_median_price_per_bed, co4_median_price_per_bed_t ], axis=1)
df_co4_test

,postcode_area_m_y,price_per_bedroom,postcode_area_m_y,median_ppb_4mnths_ago
0,CO4_10_2018,98333.333333,CO4_02_2019,98333.333333


In [22]:
#some of the postal areas would not have sale transactions four months ago.
#such transactions will have Nan median_ppb_4mnths_ago
data_test = data_prep[(data_prep['median_ppb_4mnths_ago'].isnull())]
data_test

,unique_id,num_of_beds,price_paid,town,district,estate_type,property_type,deed_date,postcode,postcode_area_m_y,price_per_bedroom,median_ppb_4mnths_ago
73,79A74E22-03DF-1289-E053-6B04A8C01627,1,128000.0,PORTSMOUTH,PORTSMOUTH,L,S,31/08/2018,PO2 9RD,PO2_08_2018,1.280000e+05,NaN
460,919FEC05-7458-9A90-E053-6C04A8C0A300,1,172500.0,MANCHESTER,MANCHESTER,L,F,25/07/2019,M4 1LY,M4_07_2019,1.725000e+05,NaN
545,75050A84-D2F3-9A88-E053-6B04A8C02390,1,97000.0,LISKEARD,CORNWALL,F,S,20/07/2018,PL14 3QN,PL14_07_2018,9.700000e+04,NaN
579,773788C2-5C25-2CE4-E053-6C04A8C05E57,1,134500.0,BOSCASTLE,CORNWALL,F,T,16/08/2018,PL35 0AL,PL35_08_2018,1.345000e+05,NaN
783,773788C2-AD4C-2CE4-E053-6C04A8C05E57,1,189950.0,FAREHAM,WINCHESTER,F,T,17/08/2018,PO15 7HQ,PO15_08_2018,1.899500e+05,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1218938,7011B109-2063-8ED6-E053-6B04A8C075C1,8,215000.0,PORT ISAAC,CORNWALL,F,T,05/06/2018,PL29 3SR,PL29_06_2018,2.687500e+04,NaN
1219084,80E1AA98-2F1B-7BF8-E053-6C04A8C00BF2,9,2950000.0,SEAVIEW,ISLE OF WIGHT,F,D,26/09/2018,PO34 5BN,PO34_09_2018,3.277778e+05,NaN
1219149,8F1B26BD-7410-53DB-E053-6C04A8C03649,10,250000.0,BARMOUTH,GWYNEDD,F,T,10/06/2019,LL42 1NA,LL42_06_2019,2.500000e+04,NaN
1219178,8F1B26BD-70AB-53DB-E053-6C04A8C03649,10,330209.0,ABERAERON,CEREDIGION,F,T,08/07/2019,SA46 0AX,SA46_07_2019,3.302090e+04,NaN


In [23]:
#Checking how many postal areas do not have median_ppb_4mnths_ago
g_test = data_test.groupby(['postcode_area_m_y'])['unique_id'].count().reset_index()
g_test.rename(columns = {'unique_id':'num of prop transactions'}, inplace = True)
g_test


,postcode_area_m_y,num of prop transactions
0,B18_05_2019,2
1,B19_07_2018,2
2,B1_05_2019,2
3,B1_08_2019,1
4,B5_07_2019,4
...,...,...
950,WV9_07_2018,4
951,YO1_04_2019,9
952,YO51_07_2018,3
953,YO60_05_2018,1


In [25]:
  #downloading csv file with postcode_area_m_y which do not have the 3m-ago median prices per bedroom. 
  #these postcode_area_m_y to be put on a db, so an alternative value could be assigned later to these postcode_areas_m_y
  m_from='01_2018'
  m_to='02_2020'
  my_file='postcode_areas_m_y_without_prop_transact_four_months_ago_{}_{}.csv'.format(m_from, m_to)
  with open(my_file, 'w') as f:
    g_test.to_csv(my_file, index=False)
  files.download(my_file)  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>